1D Heat Transfer Analysis
Governing equation: $$\nabla(\vec{k}\nabla\theta) = -q^B$$
where $\theta$ is the temperature and $q$ is the heat.
In 1D, this is $$\theta '' = \frac{-q^B}{k}$$
The weak from of the heat equation is then: $$(\theta ',\phi ') = (\frac{-q^B}{k},\phi)$$
where $(\cdot,\cdot)$ denotes scalor product and $\phi$ is any function in $V$ which is a vector space containing all function who satisfy the boundary conditions and whose first derivative is piece-wise contenious.
Now consider $v_h$ in place of $\phi$, where $v_h$ exisits in $V_h$ which is finite dimentional (unlike $V$).
Now let us create $n$ "elements", and basis functions corresponding to the elements. For simplisity, we consider only linear functions $\phi$ with the following condition:
\begin{equation}
  \phi_i(x_j) =
    \begin{cases}
      1 & \text{if $i=j$}\\
      0 & \text{if $i\neq j$}\\
    \end{cases}       
\end{equation}
That is, $$v_h = \sum\limits_{i=0}^{n}v_i\phi_i$$
Now, we re-write our 1D heat equation in weak form: $$\sum\limits_{i=0}^{n}u_i(\phi_i',\phi_j')=(\frac{-q^B}{k},\phi_j)$$
That is: $$AU=F$$
where $A_{ij}=(\phi_i',\phi_j')$, $U^T={u_1, u_2, u_3, ... u_n}$, and $F^T_i = (-q^B, \phi_i)$
We want to solve for $U$.


In [2]:
import numpy as np
import math
import sympy as sym
import math

In [3]:
#setting the variables/functions
n_element = 10
dx = 1/n_element
#the base functions are cones with magnitude 1 at each node.
def get_base(ith_node, dx):
    x = sym.Symbol( 'x', real=True )
    formula = sym.Piecewise((x/dx+(1-ith_node), sym.Interval(dx*ith_node-dx, dx*ith_node).contains(x)), (-x/dx+(1+ith_node), sym.Interval(dx*ith_node, dx*ith_node+dx).contains(x)), (0,True))
    return formula
#matrix elements for A
def get_a(row, column):
    x = sym.Symbol('x', real = True)
    row_base = get_base(row, dx)
    column_base = get_base(column, dx)
    diff_row_base = sym.diff(row_base, x)
    diff_column_base = sym.diff(column_base, x)
    return sym.integrate(diff_row_base*diff_column_base, (x, 0, 1))
#matrix elements for F
def get_f(row):
    x = sym.Symbol('x', real = True)
    k = 1
    q = 1
    f = -q/k
    row_base = get_base(row, dx)
    return sym.integrate(f*row_base, (x, 0, 1))

In [4]:
#this one takes a while to run (>1min)
#Assembling matrix A
A = np.zeros((n_element+1, n_element+1))
for ith_row in range(1, n_element):
    A.itemset((ith_row, ith_row-1), get_a(ith_row, ith_row-1))
    A.itemset((ith_row, ith_row), get_a(ith_row, ith_row))
    A.itemset((ith_row, ith_row+1), get_a(ith_row, ith_row+1))
A.itemset((0,0), get_a(0,0))
A.itemset((0,1), get_a(0,1))
A.itemset((n_element, n_element), get_a(n_element, n_element))
A.itemset((n_element, n_element-1), get_a(n_element, n_element-1))


In [5]:
#Assumblying vector F
F = np.zeros(n_element+1)
for ith_element in range(n_element+1):
    F.itemset(ith_element, get_f(ith_element))


In [6]:
print('A=', A)
print('F=', F)

A= [[ 10. -10.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [-10.  20. -10.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0. -10.  20. -10.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0. -10.  20. -10.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0. -10.  20. -10.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0. -10.  20. -10.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0. -10.  20. -10.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0. -10.  20. -10.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0. -10.  20. -10.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0. -10.  20. -10.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0. -10.  10.]]
F= [-0.05 -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.1  -0.05]


In [8]:
A.itemset((0,1), 0)
A.itemset((1,0),0)
A.itemset((n_element, n_element-1), 0)
A.itemset((n_element-1, n_element), 0)
F.itemset(0,0)
F.itemset(10,0)
U = np.dot(np.linalg.inv(A), F)
print(U)

[ 0.    -0.045 -0.08  -0.105 -0.12  -0.125 -0.12  -0.105 -0.08  -0.045
  0.   ]


In [9]:
A.itemset((0,1), 0)
A.itemset((1,0),-10)
A.itemset((n_element, n_element-1), 0)
A.itemset((n_element-1, n_element), -10)
F.itemset(0,0)
F.itemset(10,0)
U = np.dot(np.linalg.inv(A), F)
print(U)

[ 0.    -0.045 -0.08  -0.105 -0.12  -0.125 -0.12  -0.105 -0.08  -0.045
  0.   ]
